## Data Inspection

In [10]:
# import pandas as pd

# df = pd.read_csv('diabetes_012_health_indicators_BRFSS2015.csv')
# print(df.shape)
# print(df.columns)
# print(df.info())
# print(df.describe())


In [11]:
# # Binary Target
# df['Diabetes_binary'] = df['Diabetes_012'].apply(lambda x: 1 if x > 0 else 0)


In [12]:
# import matplotlib.pyplot as plt

# df['Diabetes_binary'].value_counts(normalize=True).plot(kind='bar')
# plt.xticks([0, 1], ['No Diabetes', 'Prediabetes/Diabetes'], rotation=0)
# plt.title('Class Distribution')
# plt.ylabel('Proportion')
# plt.show()


In [13]:
# import seaborn as sns

# correlations = df.corr()['Diabetes_binary'].sort_values(ascending=False)
# print(correlations.head(10))

# top_corr_features = correlations[1:11].index
# sns.heatmap(df[top_corr_features].corr(), annot=True, cmap='coolwarm')
# plt.title('Heatmap of Top Correlated Features')
# plt.show()


In [14]:
# # Correct numerical features
# num_features = ['BMI', 'PhysHlth', 'MentHlth', 'GenHlth']

# for feature in num_features:
#     sns.boxplot(x='Diabetes_binary', y=feature, data=df)
#     plt.title(f'{feature} vs Diabetes_binary')
#     plt.xticks([0, 1], ['No Diabetes', 'Prediabetes/Diabetes'])
#     plt.show()


In [15]:
# cat_features = ['HighBP', 'HighChol', 'Smoker', 'PhysActivity', 'DiffWalk', 'Sex', 'Age', 'Education', 'Income']

# for feature in cat_features:
#     sns.countplot(x=feature, hue='Diabetes_binary', data=df)
#     plt.title(f'{feature} Distribution by Diabetes Status')
#     plt.legend(title='Diabetes', labels=['No', 'Yes'])
#     plt.show()


In [16]:
# from sklearn.feature_selection import SelectKBest, f_classif

# X = df.drop(columns=['Diabetes_012', 'Diabetes_binary'])
# y = df['Diabetes_binary']

# selector = SelectKBest(score_func=f_classif, k=10)
# selector.fit(X, y)

# scores = pd.DataFrame({
#     'Feature': X.columns,
#     'F-Score': selector.scores_
# }).sort_values(by='F-Score', ascending=False)

# print(scores)


In [17]:
# # BMI vs. Diabetes by HighBP & HighChol
# g = sns.FacetGrid(df, col='HighBP', row='HighChol', hue='Diabetes_binary', margin_titles=True)
# g.map_dataframe(sns.kdeplot, x='BMI', fill=True)
# g.add_legend()
# plt.show()


## Neural Network 2: 6-Layer with 8, 16, 21, 32 Neurons (UNCLEANED)

---

In [21]:
# ─── Cell 1: Model factory ────────────────────────────────────────────────────────────────────
from tensorflow.keras import Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

# after you’ve created X_train_norm elsewhere...
n_features = X_train_norm.shape[1]

def make_model(units):
    # Returns a compiled 6-layer neural network with `units` neurons per hidden layer
    model = Sequential([
        Input(shape=(n_features,)),        # ← dynamic feature count
        Dense(units, activation='relu'),
        Dense(units, activation='relu'),
        Dense(units, activation='relu'),
        Dense(units, activation='relu'),
        Dense(units, activation='relu'),
        Dense(units, activation='relu'),
        Dense(3, activation='softmax')     # 3 output classes
    ])

    model.compile(
        optimizer=Adam(),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model


In [22]:
# Cell 1: Load & Normalize the Data
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# 1. Load the Kaggle CSV (adjust path as needed)
df = pd.read_csv('diabetes_012_health_indicators_BRFSS2015.csv')

# 2. Split into features (X) and target (y)
X = df.drop('Diabetes_012', axis=1).values
y = df['Diabetes_012'].values

# 3. Train/test split (stratify to keep class proportions)
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

# 4. Apply Min–Max normalization so all features lie in [0,1]
scaler = MinMaxScaler()
X_train_norm = scaler.fit_transform(X_train)
X_test_norm  = scaler.transform(X_test)

# Now X_train_norm and X_test_norm are ready for your network.
print(X_train_norm.shape, y_train.shape)  # should be (202944, 21), (202944,)
print(X_test_norm.shape,  y_test.shape)   # should be (50736, 21), (50736,)
print(X_train_norm.min(), X_train_norm.max())  # should be 0.0, 1.0

(202944, 21) (202944,)
(50736, 21) (50736,)
0.0 1.0000000000000002


In [23]:
# Cell 2: 6 layer neural network with 8 neurons
#         Build, train & evaluate 8-unit model
import time 
# numpy as np
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score

model_8_neurons = make_model(8)
model_8_neurons.summary()

start = time.time()
history_8 = model_8_neurons.fit(
    X_train_norm, y_train,
    epochs=20, batch_size=32,
    validation_split=0.1, verbose=1
)
train_time_8 = time.time() - start

y_pred_8 = model_8_neurons.predict(X_test_norm).argmax(axis=1)
mse_8 = mean_squared_error(y_test, y_pred_8)
r2_8  = r2_score(y_test, y_pred_8)
acc_8 = accuracy_score(y_test, y_pred_8)

print(f"8 neurons time to train: {train_time_8:.1f}s, MSE: {mse_8:.4f}, R²: {r2_8:.4f}, ACC: {acc_8:.4f}")


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 8)              │           176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 8)              │            72 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 8)              │            72 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 8)              │            72 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 8)              │            72 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 8)              │            72 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 3)              │            27 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 563 (2.20 KB)

 Trainable params: 563 (2.20 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
5708/5708 ━━━━━━━━━━━━━━━━━━━━ 4s 557us/step - accuracy: 0.8249 - loss: 0.4826 - val_accuracy: 0.8481 - val_loss: 0.3978
Epoch 2/20
5708/5708 ━━━━━━━━━━━━━━━━━━━━ 3s 537us/step - accuracy: 0.8446 - loss: 0.4047 - val_accuracy: 0.8495 - val_loss: 0.3933
Epoch 3/20
5708/5708 ━━━━━━━━━━━━━━━━━━━━ 3s 528us/step - accuracy: 0.8457 - loss: 0.4022 - val_accuracy: 0.8484 - val_loss: 0.3920
Epoch 4/20
5708/5708 ━━━━━━━━━━━━━━━━━━━━ 3s 550us/step - accuracy: 0.8476 - loss: 0.3980 - val_accuracy: 0.8501 - val_loss: 0.3911
Epoch 5/20
5708/5708 ━━━━━━━━━━━━━━━━━━━━ 3s 536us/step - accuracy: 0.8468 - loss: 0.4000 - val_accuracy: 0.8503 - val_loss: 0.3915
Epoch 6/20
5708/5708 ━━━━━━━━━━━━━━━━━━━━ 3s 537us/step - accuracy: 0.8469 - loss: 0.4005 - val_accuracy: 0.8500 - val_loss: 0.3943
Epoch 7/20
5708/5708 ━━━━━━━━━━━━━━━━━━━━ 3s 527us/step - accuracy: 0.8482 - loss: 0.3956 - val_accuracy: 0.8502 - val_loss: 0.3903
Epoch 8/20
5708/5708 ━━━━━━━━━━━━━━━━━━━━ 3s 518us/step - accuracy: 0.8484 -

In [24]:
# Cell 2: 6 layer neural network with 16 neurons
#         Build, train & evaluate 8-unit model
import time 
# numpy as np
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score

model_16_neurons = make_model(16)
model_16_neurons.summary()

start = time.time()
history_16 = model_16_neurons.fit(
    X_train_norm, y_train,
    epochs=20, batch_size=32,
    validation_split=0.1, verbose=1
)
train_time_16 = time.time() - start

y_pred_16 = model_16_neurons.predict(X_test_norm).argmax(axis=1)
mse_16 = mean_squared_error(y_test, y_pred_16)
r2_16  = r2_score(y_test, y_pred_16)
acc_16 = accuracy_score(y_test, y_pred_16)

print(f"16 neurons time to train: {train_time_16:.1f}s, MSE: {mse_16:.4f}, R²: {r2_16:.4f}, ACC: {acc_16:.4f}")


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_7 (Dense)                 │ (None, 16)             │           352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 16)             │           272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 16)             │           272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 16)             │           272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 16)             │           272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 16)             │           272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 3)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,763 (6.89 KB)

 Trainable params: 1,763 (6.89 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
5708/5708 ━━━━━━━━━━━━━━━━━━━━ 4s 562us/step - accuracy: 0.8370 - loss: 0.4414 - val_accuracy: 0.8483 - val_loss: 0.3973
Epoch 2/20
5708/5708 ━━━━━━━━━━━━━━━━━━━━ 3s 565us/step - accuracy: 0.8442 - loss: 0.4043 - val_accuracy: 0.8486 - val_loss: 0.3926
Epoch 3/20
5708/5708 ━━━━━━━━━━━━━━━━━━━━ 3s 595us/step - accuracy: 0.8464 - loss: 0.4012 - val_accuracy: 0.8484 - val_loss: 0.3931
Epoch 4/20
5708/5708 ━━━━━━━━━━━━━━━━━━━━ 3s 598us/step - accuracy: 0.8458 - loss: 0.4004 - val_accuracy: 0.8505 - val_loss: 0.3926
Epoch 5/20
5708/5708 ━━━━━━━━━━━━━━━━━━━━ 3s 563us/step - accuracy: 0.8452 - loss: 0.3998 - val_accuracy: 0.8511 - val_loss: 0.3917
Epoch 6/20
5708/5708 ━━━━━━━━━━━━━━━━━━━━ 3s 576us/step - accuracy: 0.8489 - loss: 0.3959 - val_accuracy: 0.8418 - val_loss: 0.3993
Epoch 7/20
5708/5708 ━━━━━━━━━━━━━━━━━━━━ 3s 602us/step - accuracy: 0.8489 - loss: 0.3956 - val_accuracy: 0.8500 - val_loss: 0.3946
Epoch 8/20
5708/5708 ━━━━━━━━━━━━━━━━━━━━ 4s 635us/step - accuracy: 0.8487 -

In [25]:
# Cell 2: 6 layer neural network with 21 neurons
#         Build, train & evaluate 8-unit model
import time 
# numpy as np
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score

model_21_neurons = make_model(21)
model_21_neurons.summary()

start = time.time()
history_21 = model_21_neurons.fit(
    X_train_norm, y_train,
    epochs=20, batch_size=32,
    validation_split=0.1, verbose=1
)
train_time_21 = time.time() - start

y_pred_21 = model_16_neurons.predict(X_test_norm).argmax(axis=1)
mse_21 = mean_squared_error(y_test, y_pred_16)
r2_21  = r2_score(y_test, y_pred_16)
acc_21 = accuracy_score(y_test, y_pred_16)

print(f"21 neurons time to train: {train_time_21:.1f}s, MSE: {mse_21:.4f}, R²: {r2_21:.4f}, ACC: {acc_21:.4f}")


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_14 (Dense)                │ (None, 21)             │           462 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 21)             │           462 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 21)             │           462 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 21)             │           462 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 21)             │           462 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 21)             │           462 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ (None, 3)              │            66 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,838 (11.09 KB)

 Trainable params: 2,838 (11.09 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
5708/5708 ━━━━━━━━━━━━━━━━━━━━ 4s 576us/step - accuracy: 0.8356 - loss: 0.4446 - val_accuracy: 0.8470 - val_loss: 0.3966
Epoch 2/20
5708/5708 ━━━━━━━━━━━━━━━━━━━━ 3s 550us/step - accuracy: 0.8460 - loss: 0.4011 - val_accuracy: 0.8509 - val_loss: 0.3930
Epoch 3/20
5708/5708 ━━━━━━━━━━━━━━━━━━━━ 3s 586us/step - accuracy: 0.8482 - loss: 0.3984 - val_accuracy: 0.8504 - val_loss: 0.3904
Epoch 4/20
5708/5708 ━━━━━━━━━━━━━━━━━━━━ 3s 559us/step - accuracy: 0.8490 - loss: 0.3961 - val_accuracy: 0.8512 - val_loss: 0.3930
Epoch 5/20
5708/5708 ━━━━━━━━━━━━━━━━━━━━ 3s 574us/step - accuracy: 0.8485 - loss: 0.3967 - val_accuracy: 0.8504 - val_loss: 0.3913
Epoch 6/20
5708/5708 ━━━━━━━━━━━━━━━━━━━━ 3s 527us/step - accuracy: 0.8502 - loss: 0.3919 - val_accuracy: 0.8473 - val_loss: 0.3999
Epoch 7/20
5708/5708 ━━━━━━━━━━━━━━━━━━━━ 3s 526us/step - accuracy: 0.8493 - loss: 0.3942 - val_accuracy: 0.8519 - val_loss: 0.3918
Epoch 8/20
5708/5708 ━━━━━━━━━━━━━━━━━━━━ 3s 574us/step - accuracy: 0.8500 -

In [26]:
# Cell 2: 6 layer neural network with 32 neurons
#         Build, train & evaluate 8-unit model
import time 
# numpy as np
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score

model_32_neurons = make_model(32)
model_32_neurons.summary()

start = time.time()
history_32 = model_32_neurons.fit(
    X_train_norm, y_train,
    epochs=20, batch_size=32,
    validation_split=0.1, verbose=1
)
train_time_32 = time.time() - start

y_pred_32 = model_32_neurons.predict(X_test_norm).argmax(axis=1)
mse_32 = mean_squared_error(y_test, y_pred_32)
r2_32  = r2_score(y_test, y_pred_32)
acc_32 = accuracy_score(y_test, y_pred_32)

print(f"32 neurons time to train: {train_time_32:.1f}s, MSE: {mse_32:.4f}, R²: {r2_32:.4f}, ACC: {acc_32:.4f}")


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_21 (Dense)                │ (None, 32)             │           704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_24 (Dense)                │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_25 (Dense)                │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_26 (Dense)                │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_27 (Dense)                │ (None, 3)              │            99 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,083 (23.76 KB)

 Trainable params: 6,083 (23.76 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
5708/5708 ━━━━━━━━━━━━━━━━━━━━ 4s 572us/step - accuracy: 0.8381 - loss: 0.4350 - val_accuracy: 0.8458 - val_loss: 0.4014
Epoch 2/20
5708/5708 ━━━━━━━━━━━━━━━━━━━━ 3s 552us/step - accuracy: 0.8452 - loss: 0.4041 - val_accuracy: 0.8497 - val_loss: 0.4020
Epoch 3/20
5708/5708 ━━━━━━━━━━━━━━━━━━━━ 4s 612us/step - accuracy: 0.8477 - loss: 0.3994 - val_accuracy: 0.8505 - val_loss: 0.3906
Epoch 4/20
5708/5708 ━━━━━━━━━━━━━━━━━━━━ 4s 615us/step - accuracy: 0.8490 - loss: 0.3937 - val_accuracy: 0.8508 - val_loss: 0.3925
Epoch 5/20
5708/5708 ━━━━━━━━━━━━━━━━━━━━ 3s 583us/step - accuracy: 0.8478 - loss: 0.3964 - val_accuracy: 0.8514 - val_loss: 0.3936
Epoch 6/20
5708/5708 ━━━━━━━━━━━━━━━━━━━━ 3s 583us/step - accuracy: 0.8472 - loss: 0.3990 - val_accuracy: 0.8508 - val_loss: 0.3903
Epoch 7/20
5708/5708 ━━━━━━━━━━━━━━━━━━━━ 3s 539us/step - accuracy: 0.8499 - loss: 0.3937 - val_accuracy: 0.8516 - val_loss: 0.3895
Epoch 8/20
5708/5708 ━━━━━━━━━━━━━━━━━━━━ 3s 539us/step - accuracy: 0.8475 -

In [27]:
# Table of results
import pandas as pd

results = [
    (8,  train_time_8,  mse_8,  r2_8,  acc_8),
    (16, train_time_16, mse_16, r2_16, acc_16),
    (21, train_time_21, mse_21, r2_21, acc_21),
    (32, train_time_32, mse_32, r2_32, acc_32),
]
df = pd.DataFrame(results, columns=['Units','Time(s)','MSE','R²','Acc'])
display(df)


,Units,Time(s),MSE,R²,Acc
0,8,61.931811,0.551285,-0.131044,0.848490
1,16,67.794682,0.562953,-0.154984,0.845573
2,21,62.986608,0.562953,-0.154984,0.845573
3,32,64.998978,0.548526,-0.125383,0.849180


## Neural Network 2: 6-Layer with 8, 16, 21, 32 Neurons (CLEANED)

---

In [31]:
# Cell 1: Load & Normalize the Data
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# 1. Load the Kaggle CSV (adjust path as needed)
df = pd.read_csv('cleaned_diabetes.csv')

# 2. Split into features (X) and target (y)
X = df.drop('Diabetes_012', axis=1).values
y = df['Diabetes_012'].values

# 3. Train/test split (stratify to keep class proportions)
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

# 4. Apply Min–Max normalization so all features lie in [0,1]
scaler = MinMaxScaler()
X_train_norm = scaler.fit_transform(X_train)
X_test_norm  = scaler.transform(X_test)

# Now X_train_norm and X_test_norm are ready for your network.
print(X_train_norm.shape, y_train.shape)  # should be (202944, 21), (202944,)
print(X_test_norm.shape,  y_test.shape)   # should be (50736, 21), (50736,)
print(X_train_norm.min(), X_train_norm.max())  # should be 0.0, 1.0

(131555, 13) (131555,)
(32889, 13) (32889,)
0.0 1.0


In [32]:
# ─── Cell 2: Train & evaluate your 8-neuron model ─────────────────────────────────────────────
import time
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score

# build the model with 8 neurons per hidden layer
model_8_neurons = make_model(8)
model_8_neurons.summary()

start = time.time()
history_8 = model_8_neurons.fit(
    X_train_norm, y_train,
    epochs=20,
    batch_size=32,
    validation_split=0.1,
    verbose=1
)
train_time_8 = time.time() - start

# predict & compute metrics
y_pred_8 = model_8_neurons.predict(X_test_norm).argmax(axis=1)
mse_8 = mean_squared_error(y_test, y_pred_8)
r2_8  = r2_score(y_test, y_pred_8)
acc_8 = accuracy_score(y_test, y_pred_8)

print(f"8 neurons time to train: {train_time_8:.1f}s, MSE: {mse_8:.4f}, R²: {r2_8:.4f}, ACC: {acc_8:.4f}")


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_42 (Dense)                │ (None, 8)              │           176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_43 (Dense)                │ (None, 8)              │            72 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_44 (Dense)                │ (None, 8)              │            72 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_45 (Dense)                │ (None, 8)              │            72 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_46 (Dense)                │ (None, 8)              │            72 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_47 (Dense)                │ (None, 8)              │            72 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_48 (Dense)                │ (None, 3)              │            27 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 563 (2.20 KB)

 Trainable params: 563 (2.20 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20


ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "dense_42" is incompatible with the layer: expected axis -1 of input shape to have value 21, but received input with shape (None, 13)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(None, 13), dtype=float32)
  • training=True
  • mask=None

In [ ]:
# Cell 2: 6 layer neural network with 16 neurons
#         Build, train & evaluate 8-unit model
import time 
# numpy as np
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score

model_16_neurons = make_model(16)
model_16_neurons.summary()

start = time.time()
history_16 = model_16_neurons.fit(
    X_train_norm, y_train,
    epochs=20, batch_size=32,
    validation_split=0.1, verbose=1
)
train_time_16 = time.time() - start

y_pred_16 = model_16_neurons.predict(X_test_norm).argmax(axis=1)
mse_16 = mean_squared_error(y_test, y_pred_16)
r2_16  = r2_score(y_test, y_pred_16)
acc_16 = accuracy_score(y_test, y_pred_16)

print(f"16 neurons time to train: {train_time_16:.1f}s, MSE: {mse_16:.4f}, R²: {r2_16:.4f}, ACC: {acc_16:.4f}")


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_7 (Dense)                 │ (None, 16)             │           224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 16)             │           272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 16)             │           272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 16)             │           272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 16)             │           272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 16)             │           272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 3)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,635 (6.39 KB)

 Trainable params: 1,635 (6.39 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
3700/3700 ━━━━━━━━━━━━━━━━━━━━ 3s 610us/step - accuracy: 0.8760 - loss: 0.4024 - val_accuracy: 0.8785 - val_loss: 0.3534
Epoch 2/20
3700/3700 ━━━━━━━━━━━━━━━━━━━━ 2s 611us/step - accuracy: 0.8809 - loss: 0.3456 - val_accuracy: 0.8779 - val_loss: 0.3526
Epoch 3/20
3700/3700 ━━━━━━━━━━━━━━━━━━━━ 2s 595us/step - accuracy: 0.8813 - loss: 0.3417 - val_accuracy: 0.8787 - val_loss: 0.3493
Epoch 4/20
3700/3700 ━━━━━━━━━━━━━━━━━━━━ 2s 587us/step - accuracy: 0.8814 - loss: 0.3402 - val_accuracy: 0.8787 - val_loss: 0.3479
Epoch 5/20
3700/3700 ━━━━━━━━━━━━━━━━━━━━ 2s 575us/step - accuracy: 0.8822 - loss: 0.3371 - val_accuracy: 0.8786 - val_loss: 0.3486
Epoch 6/20
3700/3700 ━━━━━━━━━━━━━━━━━━━━ 2s 593us/step - accuracy: 0.8819 - loss: 0.3402 - val_accuracy: 0.8785 - val_loss: 0.3475
Epoch 7/20
3700/3700 ━━━━━━━━━━━━━━━━━━━━ 2s 599us/step - accuracy: 0.8810 - loss: 0.3427 - val_accuracy: 0.8784 - val_loss: 0.3467
Epoch 8/20
3700/3700 ━━━━━━━━━━━━━━━━━━━━ 2s 521us/step - accuracy: 0.8810 -

In [ ]:
# Cell 2: 6 layer neural network with 21 neurons
#         Build, train & evaluate 8-unit model
import time 
# numpy as np
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score

model_21_neurons = make_model(21)
model_21_neurons.summary()

start = time.time()
history_21 = model_21_neurons.fit(
    X_train_norm, y_train,
    epochs=20, batch_size=32,
    validation_split=0.1, verbose=1
)
train_time_21 = time.time() - start

y_pred_21 = model_16_neurons.predict(X_test_norm).argmax(axis=1)
mse_21 = mean_squared_error(y_test, y_pred_16)
r2_21  = r2_score(y_test, y_pred_16)
acc_21 = accuracy_score(y_test, y_pred_16)

print(f"21 neurons time to train: {train_time_21:.1f}s, MSE: {mse_21:.4f}, R²: {r2_21:.4f}, ACC: {acc_21:.4f}")


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_14 (Dense)                │ (None, 21)             │           294 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 21)             │           462 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 21)             │           462 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 21)             │           462 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 21)             │           462 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 21)             │           462 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ (None, 3)              │            66 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,670 (10.43 KB)

 Trainable params: 2,670 (10.43 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
3700/3700 ━━━━━━━━━━━━━━━━━━━━ 3s 540us/step - accuracy: 0.8651 - loss: 0.4181 - val_accuracy: 0.8785 - val_loss: 0.3545
Epoch 2/20
3700/3700 ━━━━━━━━━━━━━━━━━━━━ 2s 519us/step - accuracy: 0.8816 - loss: 0.3422 - val_accuracy: 0.8788 - val_loss: 0.3475
Epoch 3/20
3700/3700 ━━━━━━━━━━━━━━━━━━━━ 2s 510us/step - accuracy: 0.8810 - loss: 0.3429 - val_accuracy: 0.8787 - val_loss: 0.3500
Epoch 4/20
3700/3700 ━━━━━━━━━━━━━━━━━━━━ 2s 516us/step - accuracy: 0.8799 - loss: 0.3445 - val_accuracy: 0.8786 - val_loss: 0.3466
Epoch 5/20
3700/3700 ━━━━━━━━━━━━━━━━━━━━ 2s 520us/step - accuracy: 0.8811 - loss: 0.3418 - val_accuracy: 0.8785 - val_loss: 0.3485
Epoch 6/20
3700/3700 ━━━━━━━━━━━━━━━━━━━━ 2s 520us/step - accuracy: 0.8807 - loss: 0.3433 - val_accuracy: 0.8783 - val_loss: 0.3570
Epoch 7/20
3700/3700 ━━━━━━━━━━━━━━━━━━━━ 2s 559us/step - accuracy: 0.8825 - loss: 0.3378 - val_accuracy: 0.8786 - val_loss: 0.3479
Epoch 8/20
3700/3700 ━━━━━━━━━━━━━━━━━━━━ 2s 521us/step - accuracy: 0.8802 -

In [ ]:
# ─── Cell 3: RandomSearch over the refined space ───────────────────────────────
import keras_tuner as kt
from tensorflow.keras.callbacks import EarlyStopping

tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=20,
    directory='kt_dir',
    project_name='diabetes_mlp_refined'
)

stop_early = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

tuner.search(
    X_train, y_train,
    epochs=30,
    validation_split=0.1,
    callbacks=[stop_early],
    class_weight=class_weights,
    batch_size=hp.Choice('batch_size', [32, 64, 128]),  # include batch_size if you like
    verbose=1
)


In [ ]:
# Table of results
import pandas as pd

results = [
    (8,  train_time_8,  mse_8,  r2_8,  acc_8),
    (16, train_time_16, mse_16, r2_16, acc_16),
    (21, train_time_21, mse_21, r2_21, acc_21),
    (32, train_time_32, mse_32, r2_32, acc_32),
]
df = pd.DataFrame(results, columns=['Units','Time(s)','MSE','R²','Acc'])
display(df)


,Units,Time(s),MSE,R²,Acc
0,8,41.586077,0.432211,-0.130769,0.880750
1,16,41.644246,0.433671,-0.134588,0.880386
2,21,40.587177,0.433671,-0.134588,0.880386
3,32,45.285154,0.430995,-0.127587,0.881054


## Hyperparameter Tuning - 8 Neurons

---

In [8]:
# Cell 1: Install & Data Prep
%pip install --upgrade keras-tuner

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import class_weight
import numpy as np

# 1) Load & split
df = pd.read_csv('cleaned_diabetes.csv')
X = df.drop('Diabetes_012', axis=1).values
y = df['Diabetes_012'].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.25,
    random_state=42,
    stratify=y
)

# 2) Scale
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test  = scaler.transform(X_test)

n_features = X_train.shape[1]

# 3) Compute class weights to handle imbalance
raw_w = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weights = dict(enumerate(raw_w))
print("Class weights:", class_weights)



[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Class weights: {0: np.float64(0.37846720368239356), 1: np.float64(22.306565382528486), 2: np.float64(3.1955693742712787)}


In [9]:
# Cell 2: Model‐Builder with 4 tunable knobs

from tensorflow.keras import Sequential, Input
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import SGD, Adam

def build_model(hp):
    # 1) L2 regularization
    reg = hp.Choice('reg_lambda', [0.0, 1e-4, 1e-3])
    # 2) Learning rate
    lr  = hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])
    # 3) Dropout rate
    dr  = hp.Choice('dropout_rate', [0.0, 0.2, 0.5])
    # 4) Optimizer type
    opt_name = hp.Choice('optimizer', ['sgd', 'adam'])
    if opt_name == 'sgd':
        mo = hp.Choice('momentum', [0.0, 0.5, 0.9])
        optimizer = SGD(learning_rate=lr, momentum=mo)
    else:
        optimizer = Adam(learning_rate=lr)

    model = Sequential([ Input(shape=(n_features,)) ])
    for _ in range(6):
        model.add(Dense(8,
                        activation='relu',
                        kernel_regularizer=l2(reg)))
        if dr > 0:
            model.add(Dropout(dr))
    model.add(Dense(3, activation='softmax'))

    model.compile(
        optimizer=optimizer,
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model


In [10]:
# Cell 3: RandomizedSearch over those 4 knobs

import keras_tuner as kt
from tensorflow.keras.callbacks import EarlyStopping

tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=20,              # try 20 random combos
    executions_per_trial=1,
    directory='kt_dir',
    project_name='diabetes_mlp'
)

stop_early = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

tuner.search(
    X_train, y_train,
    epochs=30,
    validation_split=0.1,
    callbacks=[stop_early],
    class_weight=class_weights,  # rebalance classes in the loss
    verbose=1
)


Trial 20 Complete [00h 00m 10s]
val_accuracy: 0.6383168697357178

Best val_accuracy So Far: 0.8774120211601257
Total elapsed time: 00h 07m 48s


In [11]:
# Cell 4: Rebuild best model and evaluate on test set

from sklearn.metrics import classification_report, confusion_matrix

best_hp = tuner.get_best_hyperparameters(1)[0]
print("Best hyperparameters:", best_hp.values)

final_model = tuner.hypermodel.build(best_hp)
final_model.fit(
    X_train, y_train,
    epochs=30,
    validation_split=0.1,
    callbacks=[stop_early],
    class_weight=class_weights,
    verbose=1
)

y_pred = final_model.predict(X_test).argmax(axis=1)
print("\nTest set classification report:")
print(classification_report(y_test, y_pred, digits=4, zero_division=0))
print("Confusion matrix:\n", confusion_matrix(y_test, y_pred))


Best hyperparameters: {'reg_lambda': 0.0001, 'learning_rate': 0.01, 'dropout_rate': 0.5, 'optimizer': 'sgd', 'momentum': 0.9}
Epoch 1/30
3469/3469 ━━━━━━━━━━━━━━━━━━━━ 3s 614us/step - accuracy: 0.2483 - loss: 1.1244 - val_accuracy: 0.8774 - val_loss: 1.0468
Epoch 2/30
3469/3469 ━━━━━━━━━━━━━━━━━━━━ 2s 551us/step - accuracy: 0.2977 - loss: 1.1081 - val_accuracy: 0.0161 - val_loss: 1.2271
Epoch 3/30
3469/3469 ━━━━━━━━━━━━━━━━━━━━ 2s 546us/step - accuracy: 0.2338 - loss: 1.1125 - val_accuracy: 0.0161 - val_loss: 1.1295
Epoch 4/30
3469/3469 ━━━━━━━━━━━━━━━━━━━━ 2s 547us/step - accuracy: 0.2144 - loss: 1.1072 - val_accuracy: 0.0161 - val_loss: 1.1209
1285/1285 ━━━━━━━━━━━━━━━━━━━━ 0s 251us/step

Test set classification report:
              precision    recall  f1-score   support

           0     0.8808    1.0000    0.9366     36209
           1     0.0000    0.0000    0.0000       614
           2     0.0000    0.0000    0.0000      4288

    accuracy                         0.8808     41

## Double check

---